In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

sns.set(font_scale=1.8, palette='Set2')

##  Сравнение градиентного бустинга и случайного леса

### 1 Задача регрессии

Исследуем зависимость качества предсказаний градиентного бустинга и случайного леса в зависимости от числа базовых моделей на примере задаче регрессии. Для случайного леса будем использовать класс `RandomForestRegressor` библиотеки `sklearn`.

In [ ]:
housing = fetch_california_housing()
X, y = housing.data, housing.target

Разобьём данные на обучающую выборку и на валидацию, выделив на валидацию 25% данных.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
def plot_compare_estimators(estimator_labels, param_grid, train_metrics,
                            test_metrics, param_label='', metrics_label='', 
                            title=''):
    '''Функция для построения графиков зависимости целевой метрики 
    от некоторого параметра модели на обучающей и на валидационной 
    выборке.
    
    :param estimator_labels: массив названий моделей
    :param param_grid: значения исследуемого параметра
    :param train_metrics: массив, где i-ый элемент - 
        это массив значений метрики на обучающей выборке для i-ой модели
    :param test_metrics: массив, где i-ый элемент - 
        это массив значений метрики на валидационной выборке для i-ой модели
    :param param_label: названия параметра
    :param metrics_label: название метрики
    :param title: заголовок для графика
    '''
    
    plt.figure(figsize=(12, 6))
    
    for estimator_id in range(len(estimator_labels)):
        label = estimator_labels[estimator_id]
        plt.plot(
            param_grid, train_metrics[estimator_id], 
            label=f'{label} train', linewidth=3
        )
        plt.plot(
            param_grid, test_metrics[estimator_id],
            label=f'{label} test', linewidth=3
        )
       
    plt.legend()
    plt.xlabel(param_label)
    plt.ylabel(metrics_label)
    plt.title(title, fontsize=20)

Обучим случайный лес для разного числа деревьев и посчитаем `mse`.

In [ ]:
rf_mse_train = []  # метрика на трейне
rf_mse_test = []  # метрика на тесте
n_estimators_grid = range(1, 300, 10)

for n_estimators in tqdm(n_estimators_grid):
    <...>

Обучим градиентный бустинг для разного числа деревьев и посчитаем `mse`.

In [ ]:
boosting_mse_train = []
boosting_mse_test = []

for n_estimators in tqdm(n_estimators_grid):
    <...>

Построим график зависимости `mse` от количества базовых моделей.

In [ ]:
plot_compare_estimators(
    ['random forest', 'boosting'], n_estimators_grid, 
    [rf_mse_train, boosting_mse_train], 
    [rf_mse_test, boosting_mse_test],
    'Количество деревьев', 'MSE', 
    'Сравнение случайного леса и градиентного бустинга'
)

**Вывод.** <...>

Сравним времена обучения и предсказания случайного леса и градиентного бустинга с параметрами по умолчанию: 

In [ ]:
%%timeit

<...>

In [ ]:
regressor_boosting = GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
%%timeit

<...>

In [ ]:
%%timeit

<...>

In [ ]:
regressor_forest = RandomForestRegressor(n_jobs=-1).fit(X_train, y_train)

In [ ]:
%%timeit

<...>

**Вывод:** <...>

### 2 Задача классификации

Сделаем аналогичный эксперимент с датасетом для классификации рукописных цифр.

In [ ]:
letters_df = pd.read_csv('letter-recognition.data', header=None)
print('shape:', letters_df.shape)
letters_df.head()

In [ ]:
print('shape:', letters_df.shape)
letters_df.head()

In [ ]:
X = letters_df.values[:, 1:]
y = letters_df.values[:, 0]

Разобьём данные на обучающую и тестовую выборки.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Обучим случайный лес для разного числа деревьев и посчитаем метрику `accuracy`.

In [ ]:
rf_accuracy_train = []
rf_accuracy_test = []
n_estimators_grid = range(1, 300, 25)

for n_estimators in tqdm(n_estimators_grid):
    <...>

Обучим градиентный бустинг для разного числа деревьев и посчитаем метрику `accuracy`.

In [ ]:
boosting_accuracy_train = []
boosting_accuracy_test = []
n_estimators_grid = range(1, 300, 25)

for n_estimators in tqdm(n_estimators_grid):
    <...>

Построим график зависимости `accuracy` от количества базовых моделей.

In [ ]:
plot_compare_estimators(
    ['random forest', 'boosting'], n_estimators_grid, 
    [rf_accuracy_train, boosting_accuracy_train],
    [rf_accuracy_test, boosting_accuracy_test],
    'Количество деревьев', 'accuracy', 
    'Сравнение случайного леса и градиентного бустинга'
)

**Вывод.** <...>